In [1]:
import logging
import torch
import numpy as np
from transformers import AutoTokenizer, Trainer
from datasets import Dataset, ClassLabel

/home/jonhue/projects/jonhue/twitter-sentiment-classification/venv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from train_test_split import select_eval
from preprocessing import preprocess
from evaluation import evaluate
from bert import tokenize, get_BERT, compute_metrics, prepare_dataset

[nltk_data] Downloading package stopwords to /home/jonhue/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/jonhue/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jonhue/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
FULL=False
EVAL_FRAC = 0.2

MODEL = 'models/baseline'
TOKENIZER = 'bert-base-uncased'
PREPROCESSING = None

In [ ]:
df_eval = select_eval()
dataset_eval = prepare_dataset(df_eval, preprocessing=PREPROCESSING)

Casting the dataset: 100%|██████████| 4/4 [00:00<00:00, 37.98ba/s]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)
eval_tokenized = tokenize(dataset_eval, tokenizer)

KeyboardInterrupt: 

In [ ]:
model = get_BERT(MODEL, device)

In [ ]:
trainer = Trainer(model, eval_dataset=eval_tokenized, tokenizer=tokenizer)

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: index, token_type_ids, text. If index, token_type_ids, text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 40000
  Batch size = 8


: 

: 